In [39]:
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

## Load the documents

In [40]:
documents = [
    "apple apples apple apple a banana an banana ",
    "apple car a car"
]

### Tokenize


In [41]:
tokens = [doc.split() for doc in documents]
print(tokens)

[['apple', 'apples', 'apple', 'apple', 'a', 'banana', 'an', 'banana'], ['apple', 'car', 'a', 'car']]


In [46]:
# Initialize the stemmer
stemmer = PorterStemmer()

# Apply stemming to the tokens
stemmed = [
    [stemmer.stem(token) for token in doc]
    for doc in tokens
]
print(stemmed)

[['appl', 'appl', 'appl', 'appl', 'a', 'banana', 'an', 'banana'], ['appl', 'car', 'a', 'car']]


In [48]:
# remove stop-words

# Get the list of English stopwords
stop_words = set(stopwords.words('english'))

bows = [
    [token for token in doc if token.lower() not in stop_words]
    for doc in stemmed
]
bows

[['appl', 'appl', 'appl', 'appl', 'banana', 'banana'], ['appl', 'car', 'car']]

In [16]:
# Create the collection vocabulary
vocabulary = set(bowA).union(set(bowB))
print(vocabulary)

{'appl', 'car', 'banana'}


In [37]:
#Count unique words in each document, i.e. Term Frequency (TF)

# Initialize dataframe to store Term Frequencies:
tf_df = pd.DataFrame(columns=list(vocabulary))
tf_df

,appl,car,banana
